In [1]:
import sys

sys.path.append('../')

In [9]:
from qiskit.providers.fake_provider import FakeMontreal
from qiskit import Aer
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator

from molecules import compute_molecule
from measurements import compute_energy, create_circuits
from grouping import Grouping
from utils import extract_paulis

In [3]:
simulator = Aer.get_backend('aer_simulator')  # Backend for simulation
NUM_SHOTS = 2 ** 15

backend = FakeMontreal()
WC_device = backend.configuration().coupling_map
WC_device = [tuple(x) for x in WC_device]

device = QasmSimulator.from_backend(backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)

kwargs_run = {'coupling_map': coupling_map, 'noise_model': noise_model}

In [4]:
molecule_name = 'LiH'
qubit_op = compute_molecule(molecule_name)

In [5]:
labels, coeffs = extract_paulis(qubit_op)
energy_exact = 0

for i in range(len(labels)):
    label = labels[i]
    if 'X' not in label and 'Y' not in label:
        energy_exact += coeffs[i].real

print('Exact energy: {}'.format(energy_exact))

Exact energy: -0.5510251879817397


In [6]:
grouping_HEEM = Grouping(qubit_op, connectivity=WC_device, print_progress=True, method='HEEM', connected_graph=True)
grouping_HEEM.group()
circuits = create_circuits(grouping_HEEM.measurements, qubit_op.num_qubits)

Pauli graph:   0%|          | 0/100 [00:00<?, ?it/s]

Grouping entangled:   0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
counts = simulator.run(circuits, shots=NUM_SHOTS, initial_layout=grouping_HEEM.T[::-1],
                       **kwargs_run).result().get_counts()

In [8]:
energy_simulation = compute_energy(counts, labels, coeffs, grouping_HEEM.groups, grouping_HEEM.measurements, NUM_SHOTS)
print('Simulated energy: {}'.format(energy_simulation))

Computing energy:   0%|          | 0/14 [00:00<?, ?it/s]

Simulated energy: -0.5578252064351189
